In [7]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import importlib

sys.path.append(os.path.abspath(".."))

import src.models.mlp.mlp_cv_trainer as cv
import src.models.mlp.mlp_cv_trainer_emb as cv_emb
import src.utils.optuna_visualizer as opv
import src.utils.telegram as te

In [8]:
# Load data
env_path = Path.cwd().parent / ".env"
load_dotenv(dotenv_path=env_path)
url = os.environ.get("OPTUNA_STORAGE_URL")

l1_tr_df2 = pd.read_parquet("../artifacts/features/l1/l1_tr_df2.parquet")
l1_test_df2 = pd.read_parquet("../artifacts/features/l1/l1_test_df2.parquet")
l1_tr_df4 = pd.read_parquet("../artifacts/features/l1/l1_tr_df4.parquet")
l1_test_df4 = pd.read_parquet("../artifacts/features/l1/l1_test_df4.parquet")

In [ ]:
# Check tuning results
study_name = "l1_mlp_v2"

study = opv.OptunaVisualizer(study_name, url)
study.visualize_optimization()
study.print_trials_table()

In [9]:
# Create OOF and test predictions
importlib.reload(cv_emb)
params = {
    "hidden_dim1": 832,
    "hidden_dim2": 672,
    "hidden_dim3": 352,
    "hidden_dim4": 288,
    "batch_size": 864,
    "lr": 0.0032694640869831816,
    "eta_min": 0.0002853657880820916,
    "dropout_rate": 0.30000000000000004,
    "activation": "ReLU"
}
trainer = cv_emb.MLPCVTrainer(**params)

oof, test_preds = trainer.fit(l1_tr_df2, l1_test_df2)
np.save("../artifacts/preds/l1/oof_single_16.npy", oof)
np.save("../artifacts/preds/l1/test_single_16.npy", test_preds)


Fold 1
Epoch 1: Train Logloss = 0.14198, Val Logloss = 0.13987
New best model saved at epoch 1, Logloss: 0.13987
Epoch 2: Train Logloss = 0.14230, Val Logloss = 0.14014
Epoch 3: Train Logloss = 0.14221, Val Logloss = 0.14007
Epoch 4: Train Logloss = 0.14166, Val Logloss = 0.13970
New best model saved at epoch 4, Logloss: 0.13970
Epoch 5: Train Logloss = 0.14328, Val Logloss = 0.14144
Epoch 6: Train Logloss = 0.14131, Val Logloss = 0.13923
New best model saved at epoch 6, Logloss: 0.13923
Epoch 7: Train Logloss = 0.14112, Val Logloss = 0.13911
New best model saved at epoch 7, Logloss: 0.13911
Epoch 8: Train Logloss = 0.14106, Val Logloss = 0.13908
New best model saved at epoch 8, Logloss: 0.13908
Epoch 9: Train Logloss = 0.14143, Val Logloss = 0.13945
Epoch 10: Train Logloss = 0.14115, Val Logloss = 0.13917
Epoch 11: Train Logloss = 0.14094, Val Logloss = 0.13899
New best model saved at epoch 11, Logloss: 0.13899
Epoch 12: Train Logloss = 0.14126, Val Logloss = 0.13931
Epoch 13: Train 

In [10]:
# Create OOF and test predictions
importlib.reload(cv_emb)
params = {
    "hidden_dim1": 832,
    "hidden_dim2": 640,
    "hidden_dim3": 416,
    "hidden_dim4": 288,
    "batch_size": 928,
    "lr": 0.003069918203113619,
    "eta_min": 0.0002911833217431107,
    "dropout_rate": 0.2,
    "activation": "ReLU"
}
trainer = cv_emb.MLPCVTrainer(**params)

oof, test_preds = trainer.fit(l1_tr_df2, l1_test_df2)
np.save("../artifacts/preds/l1/oof_single_17.npy", oof)
np.save("../artifacts/preds/l1/test_single_17.npy", test_preds)


Fold 1
Epoch 1: Train Logloss = 0.14182, Val Logloss = 0.13982
New best model saved at epoch 1, Logloss: 0.13982
Epoch 2: Train Logloss = 0.14254, Val Logloss = 0.14033
Epoch 3: Train Logloss = 0.14162, Val Logloss = 0.13960
New best model saved at epoch 3, Logloss: 0.13960
Epoch 4: Train Logloss = 0.14222, Val Logloss = 0.14006
Epoch 5: Train Logloss = 0.14132, Val Logloss = 0.13928
New best model saved at epoch 5, Logloss: 0.13928
Epoch 6: Train Logloss = 0.14124, Val Logloss = 0.13923
New best model saved at epoch 6, Logloss: 0.13923
Epoch 7: Train Logloss = 0.14121, Val Logloss = 0.13930
Epoch 8: Train Logloss = 0.14130, Val Logloss = 0.13931
Epoch 9: Train Logloss = 0.14110, Val Logloss = 0.13912
New best model saved at epoch 9, Logloss: 0.13912
Epoch 10: Train Logloss = 0.14118, Val Logloss = 0.13923
Epoch 11: Train Logloss = 0.14117, Val Logloss = 0.13918
Epoch 12: Train Logloss = 0.14181, Val Logloss = 0.13981
Epoch 13: Train Logloss = 0.14115, Val Logloss = 0.13923
Epoch 14: 

In [11]:
# Create OOF and test predictions
importlib.reload(cv_emb)
params = {
    "hidden_dim1": 896,
    "hidden_dim2": 288,
    "hidden_dim3": 96,
    "hidden_dim4": 32,
    "batch_size": 704,
    "lr": 0.01120760621186057,
    "eta_min": 0.0002703616066662003,
    "dropout_rate": 0.2,
    "activation": "ReLU"
}
trainer = cv_emb.MLPCVTrainer(**params)

oof, test_preds = trainer.fit(l1_tr_df2, l1_test_df2)
np.save("../artifacts/preds/l1/oof_single_18.npy", oof)
np.save("../artifacts/preds/l1/test_single_18.npy", test_preds)


Fold 1
Epoch 1: Train Logloss = 0.14220, Val Logloss = 0.14001
New best model saved at epoch 1, Logloss: 0.14001
Epoch 2: Train Logloss = 0.14159, Val Logloss = 0.13951
New best model saved at epoch 2, Logloss: 0.13951
Epoch 3: Train Logloss = 0.14169, Val Logloss = 0.13968
Epoch 4: Train Logloss = 0.14218, Val Logloss = 0.14025
Epoch 5: Train Logloss = 0.14146, Val Logloss = 0.13952
Epoch 6: Train Logloss = 0.14180, Val Logloss = 0.13980
Epoch 7: Train Logloss = 0.14130, Val Logloss = 0.13937
New best model saved at epoch 7, Logloss: 0.13937
Epoch 8: Train Logloss = 0.14133, Val Logloss = 0.13931
New best model saved at epoch 8, Logloss: 0.13931
Epoch 9: Train Logloss = 0.14147, Val Logloss = 0.13937
Epoch 10: Train Logloss = 0.14138, Val Logloss = 0.13931
Epoch 11: Train Logloss = 0.14107, Val Logloss = 0.13905
New best model saved at epoch 11, Logloss: 0.13905
Epoch 12: Train Logloss = 0.14138, Val Logloss = 0.13946
Epoch 13: Train Logloss = 0.14125, Val Logloss = 0.13929
Epoch 14:

In [12]:
# Create OOF and test predictions
importlib.reload(cv_emb)
params = {
    "hidden_dim1": 672,
    "hidden_dim2": 352,
    "hidden_dim3": 64,
    "hidden_dim4": 32,
    "batch_size": 512,
    "lr": 0.0187422109855557,
    "eta_min": 0.00020623196560647133,
    "dropout_rate": 0.25,
    "activation": "ReLU"
}
trainer = cv_emb.MLPCVTrainer(**params)

oof, test_preds = trainer.fit(l1_tr_df4, l1_test_df4)
np.save("../artifacts/preds/l1/oof_single_19.npy", oof)
np.save("../artifacts/preds/l1/test_single_19.npy", test_preds)


Fold 1
Epoch 1: Train Logloss = 0.14148, Val Logloss = 0.13949
New best model saved at epoch 1, Logloss: 0.13949
Epoch 2: Train Logloss = 0.14242, Val Logloss = 0.14048
Epoch 3: Train Logloss = 0.14202, Val Logloss = 0.14022
Epoch 4: Train Logloss = 0.14230, Val Logloss = 0.14027
Epoch 5: Train Logloss = 0.14146, Val Logloss = 0.13952
Epoch 6: Train Logloss = 0.14199, Val Logloss = 0.13993
Epoch 7: Train Logloss = 0.14116, Val Logloss = 0.13917
New best model saved at epoch 7, Logloss: 0.13917
Epoch 8: Train Logloss = 0.14133, Val Logloss = 0.13939
Epoch 9: Train Logloss = 0.14232, Val Logloss = 0.14021
Epoch 10: Train Logloss = 0.14126, Val Logloss = 0.13925
Epoch 11: Train Logloss = 0.14147, Val Logloss = 0.13941
Epoch 12: Train Logloss = 0.14086, Val Logloss = 0.13897
New best model saved at epoch 12, Logloss: 0.13897
Epoch 13: Train Logloss = 0.14145, Val Logloss = 0.13942
Epoch 14: Train Logloss = 0.14091, Val Logloss = 0.13905
Epoch 15: Train Logloss = 0.14116, Val Logloss = 0.1

In [13]:
# Create OOF and test predictions
importlib.reload(cv_emb)
params = {
    "hidden_dim1": 480,
    "hidden_dim2": 224,
    "hidden_dim3": 192,
    "hidden_dim4": 128,
    "batch_size": 1056,
    "lr": 0.008798929749689027,
    "eta_min": 0.00013170256885255112,
    "dropout_rate": 0.30000000000000004,
    "activation": "GELU"
}
trainer = cv_emb.MLPCVTrainer(**params)

oof, test_preds = trainer.fit(l1_tr_df4, l1_test_df4)
np.save("../artifacts/preds/l1/oof_single_20.npy", oof)
np.save("../artifacts/preds/l1/test_single_20.npy", test_preds)


Fold 1
Epoch 1: Train Logloss = 0.14180, Val Logloss = 0.14002
New best model saved at epoch 1, Logloss: 0.14002
Epoch 2: Train Logloss = 0.14136, Val Logloss = 0.13952
New best model saved at epoch 2, Logloss: 0.13952
Epoch 3: Train Logloss = 0.14134, Val Logloss = 0.13944
New best model saved at epoch 3, Logloss: 0.13944
Epoch 4: Train Logloss = 0.14172, Val Logloss = 0.13983
Epoch 5: Train Logloss = 0.14103, Val Logloss = 0.13907
New best model saved at epoch 5, Logloss: 0.13907
Epoch 6: Train Logloss = 0.14094, Val Logloss = 0.13901
New best model saved at epoch 6, Logloss: 0.13901
Epoch 7: Train Logloss = 0.14163, Val Logloss = 0.13985
Epoch 8: Train Logloss = 0.14138, Val Logloss = 0.13960
Epoch 9: Train Logloss = 0.14090, Val Logloss = 0.13904
Epoch 10: Train Logloss = 0.14113, Val Logloss = 0.13914
Epoch 11: Train Logloss = 0.14084, Val Logloss = 0.13896
New best model saved at epoch 11, Logloss: 0.13896
Epoch 12: Train Logloss = 0.14082, Val Logloss = 0.13894
New best model s

In [14]:
# Create OOF and test predictions
importlib.reload(cv_emb)
params = {
    "hidden_dim1": 448,
    "hidden_dim2": 352,
    "hidden_dim3": 224,
    "hidden_dim4": 128,
    "batch_size": 864,
    "lr": 0.006572408995494359,
    "eta_min": 0.0001368560696479014,
    "dropout_rate": 0.15000000000000002,
    "activation": "GELU"
}
trainer = cv_emb.MLPCVTrainer(**params)

oof, test_preds = trainer.fit(l1_tr_df4, l1_test_df4)
np.save("../artifacts/preds/l1/oof_single_21.npy", oof)
np.save("../artifacts/preds/l1/test_single_21.npy", test_preds)


Fold 1
Epoch 1: Train Logloss = 0.14194, Val Logloss = 0.14008
New best model saved at epoch 1, Logloss: 0.14008
Epoch 2: Train Logloss = 0.14173, Val Logloss = 0.13970
New best model saved at epoch 2, Logloss: 0.13970
Epoch 3: Train Logloss = 0.14154, Val Logloss = 0.13948
New best model saved at epoch 3, Logloss: 0.13948
Epoch 4: Train Logloss = 0.14089, Val Logloss = 0.13892
New best model saved at epoch 4, Logloss: 0.13892
Epoch 5: Train Logloss = 0.14195, Val Logloss = 0.13995
Epoch 6: Train Logloss = 0.14122, Val Logloss = 0.13943
Epoch 7: Train Logloss = 0.14177, Val Logloss = 0.13985
Epoch 8: Train Logloss = 0.14099, Val Logloss = 0.13901
Epoch 9: Train Logloss = 0.14133, Val Logloss = 0.13939
Epoch 10: Train Logloss = 0.14136, Val Logloss = 0.13943
Epoch 11: Train Logloss = 0.14119, Val Logloss = 0.13920
Epoch 12: Train Logloss = 0.14099, Val Logloss = 0.13906
Epoch 13: Train Logloss = 0.14125, Val Logloss = 0.13939
Epoch 14: Train Logloss = 0.14092, Val Logloss = 0.13900
Epo

In [15]:
te.send_telegram_message("MLP Training complete!")

✅ Message sent successfully.
